# Welcome to the Tracking notebook using trackastra

In this notebook, we showcase the single-cell tracking of a time-lapse using [trackastra](https://doi.org/10.1007/978-3-031-73116-7_27) within the [acia](https://pypi.org/project/acia/) framework.

Therfore, we perform the following steps:

1. Install software dependencies
2. Download live-cell imaging data
3. Visualize general information about the image stack
4. Perform deep-learning based tracking using [trackastra](https://doi.org/10.1007/978-3-031-73116-7_27)
5. Visualize tracking result

Enjoy the automated analysis and customize the notebook where you need it to introduce custom visualizations or computations. Have fun 🚀

# 1. ⚙️ Install software dependencies

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport acia

In [ ]:
# install acia dependencies
%pip uninstall acia -y
%pip install acia==0.3.1

# dependencies for trackastra
%pip uninstall -y trackastra
%pip install "trackastra==0.3.2"

In [ ]:
import torch
import logging

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

cuda = torch.cuda.is_available()

if not cuda:
  logging.warning("You are not using GPU computation. Thus the deep learning segmentation might take a while!")
  if IN_COLAB:
    logging.warning("Please go to 'Runtime > Change runtime type' in order to select a GPU based runtime in colab!")

In [ ]:
import os
from pathlib import Path

# get the acia unit registry
from acia import ureg

# id of the image set (here it is a path to the folder)
image_id = "data/toiam-1-sequence/00" # change the id if you want to apply the analysis to different image data
mask_id = "data/toiam-1-sequence/00_RES"

# channel of the phase-contrast images (chanel ordering starting with 0...)
phase_contrast_channel = 0

# size of a single pixel in the image
pixel_size = 0.072 * ureg.micrometer

# higher values than 1 are used to reduce the computational burden (especially segmentation)
subsampling_factor = 5

# the number of images used for the analysis
num_images = 500

# framerate of the video
framerate=20

# use current working directory as default storage folder for outputs
storage_folder = os.getcwd()

## Parameters

In [ ]:
from pathlib import Path

if subsampling_factor > 1:
  logging.warning("You are using subsampling. This will accelerate the analysis but can lead to imprecise results")

# the imaging interval of the recorded time-lapse
imaging_interval = 1 * ureg.minute * subsampling_factor

# create the output directory
output_path = Path(storage_folder) / "output/"
output_path.mkdir(parents=True, exist_ok=True)

# make path relative (advantage in video embedding)
output_path_rel = output_path.relative_to(Path(os.getcwd()))

image_id = Path(image_id)

# 2. 💾 Download live-cell imaging data

In [ ]:
from pathlib import Path
if not Path(image_id).exists():
  !wget -O data.zip https://fz-juelich.sciebo.de/s/D0A9ftcpqwKm1Rq/download
  !unzip -q data.zip -d data/
#!unzip data.zip -d data/

In [ ]:
from pathlib import Path
import tifffile
from acia.segm.local import THWCSequenceSource
import numpy as np
from tqdm.auto import tqdm
from acia.segm.formats import overlay_from_masks

image_files = sorted(Path(image_id).glob("t*.tif"))
image_stack = np.stack([tifffile.imread(im_file) for im_file in tqdm(image_files[:num_images][::subsampling_factor], desc="Loading images...")]).astype(np.uint8)

# bring the image stack into TxHxWxC (time, height, width, channels) format
source = THWCSequenceSource(image_stack[...,None])

# Load the segmentation masks
mask_files = sorted(Path(mask_id).glob("mask*.tif"))
mask_stack = np.stack([tifffile.imread(im_file) for im_file in tqdm(mask_files[:num_images][::subsampling_factor], desc="Loading masks...")])

overlay = overlay_from_masks(mask_stack)

# 3. Information about the image stack

In [ ]:
import matplotlib.pyplot as plt

T = source.size_t
C = source.size_c

# display markdown
from IPython.display import Video, Markdown, display
display(Markdown("# Image information"))

table = f"""
| Value    | Content |
| --- | --- |
| Image Path | {image_id} |
| T Size | { T } |
| C Size | { C } |
| Channels | {','.join([f"{c}" for c in range(C)])} |
| Imaging Interval | {imaging_interval} |
| Pixel Size | {pixel_size} |
| Phase-Contrast Channel | {phase_contrast_channel} |
| Image dtype | {image_stack.dtype}
"""

display(Markdown(table))
display(Markdown(f"## Preview of channels"))

t = T // 2

image = source.get_frame(t).raw

fig, ax = plt.subplots(1, C, figsize=(15, 15))
for i, c in enumerate(range(0, C)):       # Channel index starts at 1

    if C > 1:
        loc_ax = ax[i]
    else:
        loc_ax = ax

    loc_ax.imshow(image[...,c], cmap="gray")
    loc_ax.set_title(f"Channel {i}, t: {t}")

plt.tight_layout()

# 4. 🦠 Cell Tracking

During cell tracking we link the cell detections through time to build a so-called lineage tree 🌴 This lineage tree contains the information about cell division and links cells through generations. For the tracking, we utilize [**trackastra**](https://link.springer.com/chapter/10.1007/978-3-031-73116-7_27).

In [ ]:
from acia.tracking.processor.trackastra import TrackastraTracker

tt = TrackastraTracker()

print("Perform tracking...")
tracking_ov, tracklet_graph, tracking_graph = tt(source, overlay)

# 5. Visualize the cell tracking

In [ ]:
import acia
from acia.segm.output import renderVideo
from acia.viz import render_tracking_mask, render_tracking, render_video, render_time, render_scalebar
import numpy as np
from acia import ureg

# video rendering configuration
video_config = dict(codec="vp9", ffmpeg_params = ["-crf", "30", "-b:v", "0", "-speed", "1"])

# scalebar placement
scalebar_config = dict(
    xy_position=(750, 1050),
    size_of_pixel = pixel_size,
    bar_width=10 * ureg.micrometer, # width of the scalebar
    bar_height="1 micrometer" # height of the scalebar
)

# timestamp placement
time_config = dict(
    xy_position=(800, 50),
    timepoints=np.array(range(num_images)) * imaging_interval, # timepoints of the individual frames (with correct unit)
    background_color = (0, 0, 0),
)

# Make a video with
video_file = str(output_path_rel / "tracked.mp4")

# do the different rendering steps sequentially
source_rend = render_time(source.to_rgb(), **time_config)
source_rend = render_scalebar(source_rend, **scalebar_config)
source_rend = render_tracking_mask(source_rend, tracking_ov, alpha=0.5)
source_rend = render_tracking(source_rend, tracking_ov, tracking_graph)

# render the output video
render_video(source_rend, filename=video_file, **video_config, framerate=framerate)

# Display the rendered segmentation
from IPython.display import Video, Markdown, display
display(Markdown("# Your tracking"))

from moviepy.editor import *
myvideo =  VideoFileClip(video_file)
myvideo.ipython_display(maxduration=400)

In [ ]:
from acia.analysis import ExtractorExecutor, AreaEx, IdEx, FrameEx, TimeEx, PositionEx, LabelEx
from acia import ureg
import numpy as np

ex = ExtractorExecutor()

df_track = ex.execute(tracking_ov, source, [
    # define the cell properties that you want to extract here
    LabelEx(),
    AreaEx(input_unit=pixel_size ** 2),  # pass the correct area of pixels
    PositionEx(input_unit=pixel_size),
    FrameEx(),
    TimeEx(input_unit=imaging_interval),  # 1/6 = one picture every 10 min, 1/60 = every 1 minutes, 1/360 = every 10 seconds
])

# add time and label information
for n in tracking_graph:
    tracking_graph.nodes[n]["time"] = df_track.loc[n]["time"]
    tracking_graph.nodes[n]["label"] = df_track.loc[n]["label"]

In [ ]:
from acia.viz import plotly_cell_lineage

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:

# Plotly interactive plot
fig = plotly_cell_lineage(
    tracking_graph, orientation='horizontal', time_feature='time',
    show_label=False, label_name='label',
    node_marker="circle", node_ms=5,
    line_color="blue", line_width=2,
    mark_births=True, birth_color="orange", birth_marker=">", birth_ms=7,
    mark_ends=True, end_color="black", end_marker='s', end_ms=7,
    figure_title="Plotly interactive lineage"
)
fig.show()

## 🔁 Reproducibility Information

pip and conda environment details

In [ ]:
%pip freeze

In [ ]:
%mamba env export